### Introduction

This notebook explores the concept of **distributional similarity**. The underlying idea is that words that appear in similar contexts tend to have similar meanings. We will build high-dimensional, sparse vector representations for words based on the contexts they appear in. These vectors will then be used to find similar words and interpret the specific contexts that contribute to their similarity.

This notebook explores distribitional simliarity in a dataset of 10,000 Wikipedia articles (4.4M words), building high-dimensional, sparse representations for words from the distinct contexts they appear in.  These representations allow for analysis of the most similar words to a given query, and are interpretable with respect to the specific contexts that are most important for determining that two words are similar.

### 1. Setup and Imports

First, we import the necessary Python libraries. 
- `collections.defaultdict`: A dictionary that provides a default value for a non-existent key.
- `collections.Counter`: A dictionary subclass for counting hashable objects.
- `math`: Provides access to basic mathematical functions.
- `operator`: Provides functions corresponding to Python's intrinsic operators (e.g., for sorting).

In [1]:
# A dictionary that assigns a default value (e.g., 0) to a new key
from collections import defaultdict, Counter
# For mathematical operations like square root
import math
# Used for sorting dictionaries by their values
import operator
# Not used in this version, but can be useful for reading compressed files
import gzip

### 2. Configuration

We set two global parameters:
- `window`: Defines the size of the context window around a target word. A window of 2 means we look at the 2 words to the left and 2 words to the right.
- `vocabSize`: The number of most frequent words for which we will create vector representations.

In [2]:
# The number of words to the left and right to be considered as context
window=2
# We will only create representations for the 10,000 most common words
vocabSize=10000

### 3. Data Loading

We load the text data from a file. The text is converted to lowercase and split into a list of words (tokens) to make processing easier.

In [3]:
# Path to the dataset
filename="../data/wiki.10K.txt"

# Open the file, read its content, convert to lowercase, and split into a list of words
wiki_data=open(filename, encoding="utf-8").read().lower().split(" ")


### 4. Vocabulary Creation

The `create_vocab` function identifies the `vocabSize` most frequent words in the dataset. It then initializes a dictionary called `word_representations`. Each key in this dictionary is one of the top words, and its value is an empty `defaultdict`. This inner dictionary will eventually store the context counts for that word.

In [4]:
# We'll only create word representation for the most frequent K words

def create_vocab(data):
    # Initialize the main dictionary to hold our word vectors
    word_representations={}
    # Use Counter to count the frequency of each word in the data
    vocab=Counter()
    for i, word in enumerate(data):
        vocab[word]+=1

    # Get a list of the most common words, limited by vocabSize
    topK=[k for k,v in vocab.most_common(vocabSize)]
    # For each of the top words, initialize its representation as an empty defaultdict
    # This inner dict will map context -> count
    for k in topK:
        word_representations[k]=defaultdict(float)
    return word_representations

### 5. Context Counting Strategies

The notebook defines two different ways to count the "context" of a word.

#### 5.1 Unigram Context (`count_unigram_context`)

This function defines the context as a "bag of words". It iterates through the data, and for each word in our vocabulary, it looks at the words in the surrounding window. It then increments the count for each individual neighboring word in the target word's context vector. The position or order of the context words is ignored.

In [5]:
# word representation for a word = its unigram distributional context (the unigrams that show
# up in a window before and after its occurence)

def count_unigram_context(data, word_representations):
    # Iterate through each word in the dataset with its index
    for i, word in enumerate(data):
        # Only process words that are in our target vocabulary
        if word not in word_representations:
            continue
        # Define the start of the context window, ensuring it doesn't go below 0
        start=i-window if i-window > 0 else 0
        # Define the end of the context window, ensuring it doesn't exceed the data length
        end=i+window+1 if i+window+1 < len(data) else len(data)
        # Iterate through the context window
        for j in range(start, end):
            # Make sure we don't count the word itself as its own context
            if i != j:
                # Increment the count for the context word (data[j]) for the target word (word)
                word_representations[word][data[j]]+=1

#### 5.2 Directional Context (`count_directional_context`)

This function uses a more sophisticated definition of context. Instead of treating context words individually, it captures the entire sequence of words to the left and to the right of the target word. It also prepends "L:" or "R:" to distinguish between left and right contexts. This captures word order and direction, creating more specific and potentially more informative context features.

In [6]:
def count_directional_context(data, word_representations):
    # Iterate through each word in the dataset with its index
    for i, word in enumerate(data):
        # Only process words that are in our target vocabulary
        if word not in word_representations:
            continue
        # Define the start and end of the context window
        start=i-window if i-window > 0 else 0
        end=i+window+1 if i+window+1 < len(data) else len(data)
        
        # Create the left context string: all words from 'start' to the target word 'i'
        left="L: %s" % ' '.join(data[start:i])
        # Create the right context string: all words from after 'i' to the 'end'
        right="R: %s" % ' '.join(data[i+1:end])
        
        # Increment the count for the left and right context strings
        word_representations[word][left]+=1
        word_representations[word][right]+=1

### 6. Vector Normalization

The `normalize` function converts each word's context count vector into a unit vector (a vector with a length or L2 norm of 1). This is a crucial step for using cosine similarity. When vectors are normalized, their dot product is mathematically equivalent to their cosine similarity, which simplifies the calculation significantly. 

$$ \cos(\theta) = {A \cdot B \over ||A|| ||B||} $$

If `||A||` and `||B||` are both 1, then:

$$ \cos(\theta) = A \cdot B $$

In [7]:
# normalize a word represenatation vector that its L2 norm is 1.
# we do this so that the cosine similarity reduces to a simple dot product

def normalize(word_representations):
    # Iterate through each word in our vocabulary
    for word in word_representations:
        total=0
        # Calculate the sum of the squares of the context counts (the squared magnitude)
        for key in word_representations[word]:
            total+=word_representations[word][key]*word_representations[word][key]
            
        # The L2 norm is the square root of the sum of squares
        total=math.sqrt(total)
        
        # If the total is zero (word has no context), skip to avoid division by zero
        if total == 0:
            continue
            
        # Divide each context count by the L2 norm to create a unit vector
        for key in word_representations[word]:
            word_representations[word][key]/=total
        

### 7. Similarity Calculation

The following functions implement the cosine similarity logic.

#### 7.1 Dot Product

This function, `dictionary_dot_product`, calculates the dot product between two sparse vectors that are represented as dictionaries. It iterates through the keys (contexts) of the first dictionary and, if the same key exists in the second, multiplies their values and adds the result to the total.

In [8]:
def dictionary_dot_product(dict1, dict2):
    dot=0
    # Iterate through the context keys in the first word's vector
    for key in dict1:
        # If the same context exists in the second word's vector
        if key in dict2:
            # Multiply their values and add to the dot product
            dot+=dict1[key]*dict2[key]
    return dot

#### 7.2 Finding All Similarities

The `find_sim` function takes a query word and calculates its cosine similarity (via dot product, since vectors are normalized) with every other word in the vocabulary.

In [9]:
def find_sim(word_representations, query):
    # Check if the query word is in our vocabulary
    if query not in word_representations:
        print("'%s' is not in vocabulary" % query)
        return None
    
    # Dictionary to store similarity scores for all other words
    scores={}
    # Iterate through every word in the vocabulary
    for word in word_representations:
        # Calculate the cosine similarity (dot product of normalized vectors)
        cosine=dictionary_dot_product(word_representations[query], word_representations[word])
        # Store the score
        scores[word]=cosine
    return scores

#### 7.3 Finding Nearest Neighbors

This function, `find_nearest_neighbors`, ties everything together. It uses `find_sim` to get all similarity scores for a query word, then sorts them in descending order to find the `K` most similar words (the "nearest neighbors") and prints them.

In [10]:
# Find the K words with highest cosine similarity to a query in a set of word_representations
def find_nearest_neighbors(word_representations, query, K):
    # Get similarity scores for the query against all words
    scores=find_sim(word_representations, query)
    # If scores were calculated successfully
    if scores != None:
        # Sort the scores dictionary by value in descending order
        sorted_x = sorted(scores.items(), key=operator.itemgetter(1), reverse=True)
        # Print the top K most similar words and their scores
        for idx, (k, v) in enumerate(sorted_x[:K]):
            print("%s\t%s\t%.5f" % (idx,k,v))

### 8. Initial Model Run (Without TF-IDF)

Now we run the pipeline using the `count_directional_context` strategy. We create the vocabulary, count the contexts, and normalize the vectors.

Explore the difference between `count_unigram_context` and `count_directional_context` for determining what counts as "context".  `count_unigram_context` counts an individual unigram in the bag of words around a target as a "context" variable, while `count_directional_context` counts the sequence of words before and after the word as a single "context"--and specifies the direction it occurs (to the left or right of the word).

In [11]:
# Step 1: Initialize the vocabulary and word representation structure
word_representations=create_vocab(wiki_data)
# Step 2: Populate the structure with directional context counts
count_directional_context(wiki_data, word_representations)
# Step 3: Normalize the context vectors for cosine similarity calculation
normalize(word_representations)

#### 8.1 Nearest Neighbors for "actor"

Let's test our model by finding the nearest neighbors for the word "actor". The results show other professions, which is a good sign that the model is capturing semantic relationships.

In [12]:
find_nearest_neighbors(word_representations, "actor", 10)

0	actor	1.00000
1	politician	0.54099
2	actress	0.52242
3	cricketer	0.42361
4	artist	0.40005
5	writer	0.38234
6	cyclist	0.36833
7	musician	0.33385
8	diplomat	0.32010
9	poet	0.31124


### 9. Interpreting Similarity

The `find_shared_contexts` function helps us understand *why* two words are considered similar. It finds the common contexts between two words and calculates how much each shared context contributes to their total cosine similarity score (which is the product of their values in the normalized vectors). It then prints the contexts with the highest contribution.

In [13]:
# Let's find the contexts shared between two words that have the most contribution
# to the cosine similarity

def find_shared_contexts(word_representations, query1, query2, K):
    # Check if the first query word is in our vocabulary
    if query1 not in word_representations:
        print("'%s' is not in vocabulary" % query1)
        return None
    
    # Check if the second query word is in our vocabulary
    if query2 not in word_representations:
        print("'%s' is not in vocabulary" % query2)
        return None
    
    # Dictionary to store the contribution of each shared context
    context_scores={}
    dict1=word_representations[query1]
    dict2=word_representations[query2]
    
    # Iterate through the contexts of the first word
    for key in dict1:
        # If the context is also present for the second word
        if key in dict2:
            # The contribution to the dot product is the product of their values
            score=dict1[key]*dict2[key]
            context_scores[key]=score

    # Sort the contexts by their contribution score in descending order
    sorted_x = sorted(context_scores.items(), key=operator.itemgetter(1), reverse=True)
    # Print the top K shared contexts
    for idx, (k, v) in enumerate(sorted_x[:K]):
        print("%s\t%s\t%.5f" % (idx,k,v))

#### 9.1 Shared Contexts for "actor" and "politician"

Here, we inspect the shared contexts between "actor" and "politician". The results show contexts like "an american", "an indian", and sentence-ending patterns. This indicates that both words are often used to describe a person's nationality or role in a similar grammatical structure.

In [14]:
find_shared_contexts(word_representations, "actor", "politician", 10)

0	R: . he	0.21961
1	L: an american	0.13391
2	R: ) .	0.11417
3	R: in the	0.01410
4	L: an indian	0.00761
5	L: a canadian	0.00677
6	L: an english	0.00564
7	R: of the	0.00564
8	L: a french	0.00423
9	R: , who	0.00338


### 10. Improving the Model with TF-IDF

A problem with raw counts is that very common contexts (e.g., being preceded by "the") dominate the similarity score but carry little semantic meaning. **TF-IDF (Term Frequency-Inverse Document Frequency)** is a weighting scheme that addresses this. It increases the weight of contexts that are frequent for a specific word (high TF) but rare across all other words (high IDF).

Q1: Fill out a function `scale_tfidf` below.  This function takes as input a dict of word_representations and scales the value for each context in `word_representations[word]` by its tf-idf score.  Use the term frequency for tf and ${N \over |\{d \in D : t \in d\}|}$ for idf.  Here, tf measure the count of a *context* term for a particular *word*, and idf measures the number of distinct *words* a particular *context* is seen with.  This function should modify `word_representations` in place.

#### 10.1 `scale_tfidf` Implementation

This function implements the TF-IDF scaling.
1.  **Calculate IDF**: It first iterates through all words and their contexts to build an IDF dictionary. This dictionary (`idfs`) maps each context to the number of *distinct words* it appears with.
2.  **Apply TF-IDF Weighting**: It then iterates through the word representations again. For each context of each word, it multiplies the existing value (the TF) by the IDF score. The IDF score is calculated as `log(N / idfs[context])`, where `N` is the total number of words in our vocabulary. The logarithm is used to dampen the effect of the IDF score, preventing it from becoming too overwhelmingly large.

In [15]:
def scale_tfidf(word_representations):
    # --- IDF CALCULATION ---
    # This will store the document frequency for each context (how many words it appears with)
    idfs=defaultdict(int)
    # For each word in our vocabulary
    for term in word_representations:
        # For each context of that word
        for context in word_representations[term]:
            # Increment the count for that context, signifying it appeared with one more word
            idfs[context]+=1
            
    # --- TF-IDF SCALING ---
    # Total number of documents (in our case, words in the vocabulary)
    N=len(word_representations)
    # For each word in our vocabulary
    for word in word_representations:
        # For each context associated with that word
        for key in word_representations[word]:
            # The current value is the Term Frequency (TF)
            # The IDF is log(N / number of words this context appears with)
            # We multiply the TF by the IDF score to get the final TF-IDF weight
            word_representations[word][key]*=math.log(N/idfs[key])

### 11. Model Run with TF-IDF

Now we repeat the pipeline, but this time we add the `scale_tfidf` step before normalizing the vectors. This will re-weight the context counts to emphasize more meaningful contexts.

In [16]:
# Step 1: Initialize the vocabulary and word representation structure
tf_idf_word_representations=create_vocab(wiki_data)
# Step 2: Populate the structure with directional context counts
count_directional_context(wiki_data, tf_idf_word_representations)
# Step 3: Re-weight the context counts using TF-IDF
scale_tfidf(tf_idf_word_representations)
# Step 4: Normalize the new TF-IDF vectors for cosine similarity calculation
normalize(tf_idf_word_representations)

Q2: Compare the results the results of tf-idf scaling with the non-scaled results above.  How does scaling change the quality of the nearest neighbors, or the sensibility of the significant contexts?  Provide examples to support your claims using `find_nearest_neighbors` and `find_shared_contexts` below.

### 12. Comparing Results

Now we answer Q2 by directly comparing the outputs of the two models for the query "two".

#### 12.1 Nearest Neighbors Comparison for "two"

**Observation:**
-   **Without TF-IDF (Top Output):** The nearest neighbors for "two" are a mix of semantically unrelated words like "term", "song", "church", and "company". While it finds other numbers ("three", "four"), the list is not very coherent. This is likely because all these words share common but generic grammatical contexts.
-   **With TF-IDF (Bottom Output):** The results are dramatically better. The nearest neighbors are almost exclusively other numbers ("three", "four", "five", "six", etc.).

**Conclusion:** TF-IDF scaling significantly improves the quality of nearest neighbors. By down-weighting common, non-descriptive contexts, it allows the model to focus on the more unique contexts that truly define the meaning of a word, leading to more semantically coherent results.

In [17]:
query="two"
# Find neighbors using the original, non-scaled representations
find_nearest_neighbors(word_representations, query, 10)
print()
# Find neighbors using the TF-IDF scaled representations
find_nearest_neighbors(tf_idf_word_representations, query, 10)

0	two	1.00000
1	three	0.68766
2	four	0.61315
3	term	0.54894
4	song	0.54867
5	church	0.54464
6	main	0.54448
7	company	0.54311
8	band	0.54142
9	building	0.54053

0	two	1.00000
1	three	0.42725
2	four	0.33787
3	five	0.31393
4	several	0.23219
5	six	0.22343
6	few	0.22313
7	eight	0.22192
8	seven	0.20937
9	twenty	0.18513


#### 12.2 Shared Contexts Comparison for "two" and "three"

**Observation:**
-   **Without TF-IDF (Top Output):** The most important shared contexts are extremely common, generic phrases like "L: of the", "L: . the", and "R: of the". These don't tell us much about what makes "two" and "three" similar, other than that they appear in grammatically similar places.
-   **With TF-IDF (Bottom Output):** The top shared contexts are much more descriptive and sensible. Contexts like "R: years later", "R: days later", and "L: divided into" are highly relevant to numbers. This shows that the model has learned that numbers often appear in temporal or quantitative phrases.

**Conclusion:** TF-IDF makes the significant contexts far more interpretable and meaningful. It successfully filters out grammatical noise and highlights the specific, semantically rich contexts that define the relationship between words.

In [18]:
query1="two"
query2="three"
# Find shared contexts using the original representations
find_shared_contexts(word_representations, query1, query2, 10)
print()
# Find shared contexts using the TF-IDF scaled representations
find_shared_contexts(tf_idf_word_representations, query1, query2, 10)

0	L: of the	0.15490
1	L: . the	0.09570
2	R: of the	0.04114
3	L: the first	0.03932
4	L: , and	0.03102
5	L: there are	0.03037
6	R: years later	0.02998
7	L: , the	0.02032
8	L: one of	0.01649
9	R: years ,	0.01081

0	R: years later	0.05778
1	L: there are	0.02744
2	R: days later	0.01653
3	L: one of	0.01490
4	R: years ,	0.01324
5	R: years .	0.01194
6	L: the first	0.01144
7	R: or more	0.01066
8	L: the next	0.01057
9	L: divided into	0.01024
